In [ ]:
DATE = "0530_ensemble_2_0525-29_0326"
DATA_TO_TEST = 'normal' # 'abn1' or 'abn2' or 'normal'

SCORE_VALUE = 0.1

import os
import pandas as pd

result_save_base_path = '/home/huray/workspace/ncc/results/'
result_save_path = os.path.join(result_save_base_path, DATE, DATA_TO_TEST)
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [2]:
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

/home/huray/venvs/ml/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load RetinaNet model

In [3]:
model_list = []
# model_list.append('snapshots/0305_add_aug_65+/resnet101_csv_10.h5')                     #3/5, 75ep
model_list.append('snapshots/0326_more_aug/resnet101_csv_44.h5')                        #3/26, 44ep
# model_list.append('snapshots/0228_all_malig_wo_nccnorm/resnet101_csv_55.h5')            #2/28, 55ep
# model_list.append('snapshots/0122_ncc_abn/resnet101_csv_47.h5')                         #1/22, 47ep
# model_list.append('snapshots/0322_add-inb-mass_aug-same-with-0305/resnet101_csv_72.h5') #3/22, 72ep

model_list.append('snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_29.h5')   #5/25, 29ep
# model_list.append('snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_58.h5') #5/25, 58ep
# model_list.append('snapshots/0529_medium_aug_balanced/resnet101_csv_43.h5')          #5/29, 43ep
# model_list.append('snapshots/0604_medium_aug_oneway/resnet101_csv_43.h5')              #6/4, 43ep
# model_list.append('snapshots/0604_medium_aug_oneway/resnet101_csv_46.h5')              #6/4, 46ep
# model_list.append('snapshots/0607_low_aug/resnet101_csv_46.h5')                      #6/7, 46ep
# model_list.append('snapshots/0620_stronger_aug/resnet101_csv_25.h5')              #6/20, 25ep

## Initialize data generators

In [4]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
elif DATA_TO_TEST == 'abn2':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
elif DATA_TO_TEST == 'normal':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_normal.csv'
else:
    print('WRONG DATA CSV PATH!')
    raise

# create a generator for testing data
val_generator = CSVGenerator(
    csv_path,
    '/home/huray/data/NCC/img_retinanet/class_2_onlyM.csv',
    val_image_data_generator,
    batch_size=1,
    is_testing=True,
)

test_file_df = pd.read_csv(csv_path, header=None)
num_of_test_files = len(test_file_df)
print(num_of_test_files)

353


## Run detection

In [5]:
image_with_no_detection = []
num_of_anno = 0
correct_detection = 0

In [6]:
img_index_list = list(range(0, num_of_test_files))
img_index_list_temp = img_index_list.copy()

In [7]:
for model_path in model_list:
    print('model: ', model_path)
    model = keras.models.load_model(model_path, custom_objects=custom_objects)
    
    img_index_list = img_index_list_temp.copy()
    print('remained images: ', len(img_index_list))
    
    for index in img_index_list:
        pass_this_file = False

        # load image
        image, image_path = val_generator.load_image(index, get_image_path=True)
#         print(image_path)

        # copy to draw on
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

        # preprocess image for network
        image = val_generator.preprocess_image(image)
        image, scale = val_generator.resize_image(image)
        annotations = val_generator.load_annotations(index)

        # process image
    #     start = time.time()
        _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
    #     print("processing time: ", time.time() - start)

        # compute predicted labels and scores
        predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
        scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

        # correct for image scale
        detections[0, :, :4] /= scale

        anno_center = []

        
        # visualize annotations
        for annotation in annotations:

            label = int(annotation[4])
            b = annotation[:4].astype(int)

            anno_center.append((b[0]+b[2])/2)
            anno_center.append((b[1]+b[3])/2)

            cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (50, 50, 255), 2)
            caption = "{}".format(val_generator.label_to_name(label))
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)


        # visualize detections
        number_of_proper_detection = 0
        for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
            if score < SCORE_VALUE:
                continue
                
            if label != 0: # ensemble uses older model which is trained with benign class.
                continue
                
            if pass_this_file is True:
                continue

            number_of_proper_detection += 1

            b = detections[0, idx, :4].astype(int)
            
            caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)

            correct_detection += 1

            if pass_this_file is False:
                # remove the detected image from the list
                img_index_list_temp.remove(index)
                pass_this_file = True

#                 cv2.imwrite(os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(index)+'.jpg'), draw)

            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
            cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
             

        # save the image into file
        cv2.imwrite(os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(index)+'.jpg'), draw)

        index += 1
    
    # release the memory
    del(model)
    keras.backend.clear_session()

model:  snapshots/0326_more_aug/resnet101_csv_44.h5


/home/huray/venvs/ml/lib/python3.5/site-packages/keras/models.py:274: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.
  sample_weight_mode=sample_weight_mode)


remained images:  353
model:  snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_29.h5
remained images:  259


In [8]:
# check remained images
img_index_list = img_index_list_temp.copy()

print("TOTAL IMAGES: ", num_of_test_files)
print("IMAGEs WITH NO DETECTION: ", len(img_index_list))
print("NORMAL RATIO (based on images): ", len(img_index_list)/num_of_test_files)

TOTAL IMAGES:  353
IMAGEs WITH NO DETECTION:  217
NORMAL RATIO (based on images):  0.6147308781869688


In [9]:
img_path_list = []

for img_index in img_index_list:
    image, image_path = val_generator.load_image(img_index, get_image_path=True)
    img_path_list.append(image_path)
    
case_list = []
for path in img_path_list:
    filename = path.split('/')[-1]
    case_num = filename.split('-')[1]
    case_list.append(str(case_num))

checked_case = []
for case in case_list:
    if case in checked_case:
        continue
    
    number_of_case = sum(1 if case in x else 0 for x in case_list)
    
    if number_of_case == 4:
        checked_case.append(case)
        print('NORMAL CASE: ', case)
        
print('NUMBER OF CASES DETECTED AS NORMAL: ', len(checked_case))

NORMAL CASE:  00001
NORMAL CASE:  00002
NORMAL CASE:  00003
NORMAL CASE:  00006
NORMAL CASE:  00007
NORMAL CASE:  00020
NORMAL CASE:  00026
NORMAL CASE:  00034
NORMAL CASE:  00038
NORMAL CASE:  00039
NORMAL CASE:  00041
NORMAL CASE:  00044
NORMAL CASE:  00046
NORMAL CASE:  00048
NORMAL CASE:  00052
NORMAL CASE:  00057
NORMAL CASE:  00070
NORMAL CASE:  00080
NORMAL CASE:  00081
NORMAL CASE:  00088
NUMBER OF CASES DETECTED AS NORMAL:  20
